In [ ]:
import bs4 as bs
import requests
from bs4 import BeautifulSoup as soup 
import pandas as pd
import numpy
from random import randint
from time import sleep
import re


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

# Collect hotel list

## Quarantine hotels list
Hotels with customer reviews that contain the word "quarantine".
Collected from here: https://en.tripadvisor.com.hk/Search?q=quarantine&searchSessionId=2D417296DF5780B246EEC73E707A39821650210061193ssid&searchNearby=false&geo=294217&sid=10B3B148392F4C5D87D196FC4E0025751650210149092&blockRedirect=true&ssrc=h&rf=1

## Non-Quarantine hotel list
Full hotels list collected from here:
https://en.tripadvisor.com.hk/SmartDeals-g294217-Hong_Kong-Hotel-Deals.html

Then, excluded those hotels that are in the Quarantine hotels list.

Next, randomly picked 96 hotels from this list for scraping.

# Code for scraping Travel Safe During COVID-19 section for each hotel

In [ ]:
def scrape_COVID19 (readFileName, sheetName, outputName):

    df = pd.read_excel(readFileName, sheet_name=sheetName)


    ### ============== try first two as example ===============

    #hotelNames = df['hotel name'][0:4].tolist()
    #hotelURLs = df['hotel URL'][0:4].tolist()
    #hotelIDs = df['hotel ID'][0:4].tolist()


    #### ================ run all ===============
    hotelNames = df['hotel name']
    hotelURLs = df['hotel URL']
    hotelIDs = df['hotel ID']
    COVID_Expect = []
    COVID_Note = []


    for hotelURL in hotelURLs:

        chromedriver_autoinstaller.install()  
        browser = webdriver.Chrome()
    
        browser.get(hotelURL)
        
        try:
        # click "Read More" to expand all text
            browser.implicitly_wait(10)
            browser.find_element(By.CSS_SELECTOR,'.enknx').click()
            browser.find_element(By.CSS_SELECTOR,'.daMwF').click()
            browser.find_element(By.CSS_SELECTOR,'.bKLYt > .duhwe > .pIRBV')
            

        except:
            pass

        
        pSource = soup(browser.page_source,'html.parser')
        
        Expect = "None"
        Note = "None"

        Travel_Safe_Section = pSource.find('div', class_ = 'ui_column is-12 fnAcQ fNRLc fvoLb _T MG')
        if Travel_Safe_Section is not None:
            Expect = Travel_Safe_Section.find('ul', class_ = 'fwFeg F1')
            Note =  Travel_Safe_Section.find('div', class_ = 'pIRBV _T')            
            Expect = re.sub('<[^<]+?>', ' ', str(Expect))
            Note = re.sub('<[^<]+?>', ' ', str(Note))
            
        COVID_Expect.append(Expect)
        COVID_Note.append(Note)

        browser.close()
    
        
    print('=========== Finished =========== ')
    COVID_df = pd.DataFrame({
        "hotel name": hotelNames,
        "hotel URL": hotelURLs,
        "hotel ID": hotelIDs,
        "travel safe expect": COVID_Expect,
        "travel safe note": COVID_Note
    })

    # export to csv
    COVID_df.to_csv(outputName, index=False)
    print("Saved "+outputName)

In [ ]:
scrape_COVID19('../data/HotelList_Quarantine.xlsx', 'hotels', '../output/COVID_Msg_Quarantine.csv')

=========== Finished =========== 
Saved COVID_Msg_Quarantine.csv


In [ ]:
scrape_COVID19('../data/HotelList_NonQuarantine.xlsx', 'hotels', '../output/COVID_Msg_NonQuarantine.csv')

=========== Finished =========== 
Saved COVID_Msg_NonQuarantine.csv


# Code for scraping individual reviews URL from each hotel

In [ ]:
################################################################################

df = pd.read_excel('../data/HotelList_Quarantine.xlsx', sheet_name='hotels')

# NonQuarantine
#df = pd.read_excel('HotelList_NonQuarantine.xlsx', sheet_name='hotels')

################################################################################

reviewURLs = []

#scrape one by one
#hotelURLs = [df['URL'][102]]

#scrape several at one time
#sometimes there are timeout problem in selenium, so scrape the data bit by bit instead of the whole list at once
hotelURLs = df['URL'][99:102].tolist() 

for hotelURL in hotelURLs:

    chromedriver_autoinstaller.install()  
    browser = webdriver.Chrome()

    browser.get(hotelURL) 
    
    #scrape only those reviews with the words "quarantine"
    elem = browser.find_element_by_css_selector('input.bozft')
    elem.clear()
    elem.send_keys("quarantine") #hide when scraping non-quarantine reviews
    elem.send_keys(Keys.RETURN)
    elem.send_keys(Keys.RETURN)

    # Wait a some seconds for the page to completely load
    sleep(5)
    
    elems = browser.find_elements(By.CSS_SELECTOR, ".fpMxB [href]")
    for elem in elems:   
        browser.implicitly_wait(5)
        links = elem.get_attribute('href')
        reviewURLs.append(links)


    # next page 2
    browser.find_element(By.CSS_SELECTOR, '.next').click()
    sleep(5)
    
    elems = ''
    elems = browser.find_elements(By.CSS_SELECTOR, ".fpMxB [href]")
    for elem in elems:
        browser.implicitly_wait(5)
        links = ''
        links = elem.get_attribute('href')
        reviewURLs.append(links)
    
    # next page 3
    browser.find_element(By.CSS_SELECTOR, '.next').click()
    sleep(5)
    
    elems = ''
    elems = browser.find_elements(By.CSS_SELECTOR, ".fpMxB [href]")
    for elem in elems:
        browser.implicitly_wait(5)
        links = ''
        links = elem.get_attribute('href')
        reviewURLs.append(links)

    # etc.....repeat if needed
  
    reviews_all_urls = pd.DataFrame({
        "Review_URL": reviewURLs
    })

    reviews_all_urls = reviews_all_urls.drop_duplicates()
    
    ################################################################################
    
    reviews_all_urls.to_csv('reviewsURLs_Quarantine.csv', mode='a', header=False, index=False)
    #reviews_all_urls.to_csv('reviewsURLs_NonQuarantine.csv', mode='a', header=False, index=False)
    
    ################################################################################
    
    print('saved for ' + str(hotelURL))
    
    browser.close()

print('============== Finished =================')    

# Code for scraping each customer review & hotel reply

In [ ]:
################################################################################

df_r = pd.read_excel('../data/reviewsURLs_Quarantine.xlsx', sheet_name='reviewURLs')

#NonQuarantine
#df_r = pd.read_excel('../data/reviewsURLs_NonQuarantine.xlsx', sheet_name='reviewURLs')

################################################################################

### ============== try first two as example ===============

# hotelIDs = df_r['HotelID'][0:3].tolist()
# reviewIDs = df_r['ReviewID'][0:3].tolist()
# reviewURLs = df_r['ReviewURL'][0:3].tolist()

### ============== try first two as example ===============


#### ================ run all ===============
hotelIDs = df_r['HotelID']
reviewIDs = df_r['ReviewID']
reviewURLs = df_r['ReviewURL']
userIDs = []
customer_reviewTitles = []           
customer_ratingDates = []   
customer_ratings = [] 
customer_comments = []      
hotelReply = []             
hotelReply_comments = []    
hotelReply_dates = [] 



for reviewURL in reviewURLs:
    
    chromedriver_autoinstaller.install()  
    browser = webdriver.Chrome()
    
    browser.get(reviewURL)
    
    try:
        # click "more" to show all the comment text and hotel reply
        browser.find_element(By.CSS_SELECTOR,'.moreBtn').click()
        browser.implicitly_wait(10)

    except:
        pass

        
    pSource = soup(browser.page_source,'html.parser')
    reviewContent = pSource.find('div', class_ = 'featured-review-container')
    
    # userID
    userID = reviewContent.find('div', id=re.compile('^UID'))
    userIDs.append(userID)
    
    
    # customer_reviewTitles
    customer_reviewTitle = reviewContent.find(id='HEADING').getText()
    customer_reviewTitles.append(customer_reviewTitle)
    
    
    # customer_ratingDates
    customer_ratingDate = reviewContent.find('span', class_ = 'ratingDate').getText()
    customer_ratingDates.append(customer_ratingDate)
    
    # customer_ratings
    customer_rating = reviewContent.find('span', class_ = 'ui_bubble_rating').get('class')[1]
    customer_ratings.append(customer_rating)
    
    # customer_comments
    customer_comment = reviewContent.find('span', class_ = 'fullText').getText()
    customer_comments.append(customer_comment)
    
    
    # hotelReply_comments & hotelReply
    hotelReply_comment = reviewContent.find('div', class_ = 'prw_reviews_text_summary_hsx')
    if hotelReply_comment is not None:
        hotelReply_comment = hotelReply_comment.getText()
        hotelReply_comments.append(hotelReply_comment)
        hotelReply.append('1')
    else:
        hotelReply_comments.append('NaN')
        hotelReply.append('0')
    
    
    # hotelReply_dates
    hotelReply_date = reviewContent.find('span', class_ = 'responseDate')
    if hotelReply_date is not None:
        hotelReply_date = hotelReply_date.getText()
        hotelReply_dates.append(hotelReply_date)
    else:
        hotelReply_dates.append('NaN')
    
    
    
    browser.close()
    
print('=========== Finished =========== ')

In [ ]:
scraped = pd.DataFrame({
    "UserID": userIDs,
    "hotel ID": hotelIDs,
    "review ID": reviewIDs,
    "review URL": reviewURLs,
    "customer review Title": customer_reviewTitles,      
    "customer review date": customer_ratingDates,   
    "customer review score": customer_ratings, 
    "customer review text": customer_comments,  
    "hotel reply yes or no": hotelReply,    
    "hotel reply text": hotelReply_comments,
    "hotel reply date": hotelReply_dates   
})

scraped.head()

In [ ]:
# export to csv
scraped.to_csv("../output/data_Quarantine_needClean.csv",encoding='utf-8-sig')

#NonQuarantine
#scraped.to_csv("data_NonQuarantine_needClean.csv",encoding='utf-8-sig')
print("Saved.")

### Then, manually do the data cleaning and filtering. 
#E.g. remove Chinese reviews, remove reviews that are in 2022 and those before Mar 2020, etc.
#Final cleaned dataset "../data/data_NonQuarantine.xlsx" & "../data/data_Quarantine.xlsx"